In [1]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
from unstructured.partition.html import partition_html

nest_asyncio.apply()

In [2]:
import os
os.environ["SERPER_API_KEY"] = "" # serper.dev API key
os.environ["OPENAI_API_KEY"] = ""


from crewai import Agent
from crewai import Task
from textwrap import dedent
from datetime import date

from crewai_tools import SerperDevTool
search_tool = SerperDevTool()



In [3]:
async def scrape_and_summarize_website(website):
  try: 
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(website)
        content = await page.content()
        await browser.close()

    elements = partition_html(text=content)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
        agent = Agent(
            role='Principal Researcher',
            goal='Do amazing research and summaries based on the content you are working with',
            backstory="You're a Principal Researcher at a big company and you need to do research about a given topic.",
            allow_delegation=False
        )
        task = Task(
            description='Find and summarize the latest and most relevant news on AI',
            agent=agent
        )
        summary = task.execute()
        summaries.append(summary)
    return "\n\n".join(summaries)
  except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
import html
from html.parser import HTMLParser

class HTMLTextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.text = []

    def handle_data(self, data):
        self.text.append(data)

    def get_text(self):
        return ''.join(self.text)

def extract_text_from_html(html_content):
    parser = HTMLTextExtractor()
    parser.feed(html_content)
    return parser.get_text()

In [5]:
new_loop = asyncio.new_event_loop()
asyncio.set_event_loop(new_loop)

website = "https://livebook.myidea.my.id/authenticate"
idata=new_loop.run_until_complete(scrape_and_summarize_website(website))

print(extract_text_from_html(idata))

new_loop.close()



An error occurred: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': 'Find and...h about a given topic.)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing


TypeError: can only concatenate str (not "NoneType") to str